# Day 3 - Lab 1: AI-Driven Backend Development

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 3! With our requirements and architecture defined, it's time to write code. In this lab, you will act as a senior developer guiding an AI co-pilot. Your task is to generate the full backend API for the Onboarding Tool. This involves not just generating code, but also connecting it to the live database we created yesterday, moving from a prototype to a functional, data-driven application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll set up our environment and load the `schema.sql` artifact from Day 2. This SQL file contains the `CREATE TABLE` statements that define our database structure, which is the perfect context to provide the LLM for code generation.

**Model Selection:**
For code generation, models specifically fine-tuned for coding are ideal. `gpt-4.1`, `o3`, or `codex-mini` are excellent choices. Experiment to see which one gives you the cleanest code.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the SQL schema.
- `save_artifact()`: To save the generated Python code.
- `clean_llm_output()`: To remove markdown fences from the generated code.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/schema.sql")
if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

2025-10-30 11:39:35,218 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

Follow the challenges below to build and connect your API.

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Task:** Generate all the necessary Python code for a FastAPI application, but with simple in-memory data storage for now. This allows us to generate and validate the code's structure before adding database complexity.

**Instructions:**
1.  Create a detailed prompt that asks the LLM to act as a senior Python developer.
2.  Provide the `sql_schema` as context.
3.  Instruct the LLM to generate three key components:
    * **Pydantic Models:** For API data validation (request/response bodies).
    * **FastAPI Endpoints:** Full CRUD (Create, Read, Update, Delete) endpoints for the `users` table.
    * **In-Memory Database:** A simple Python list to act as a temporary, fake database.
4.  The final output should be a single Python script for a `main_in_memory.py` file.
5.  Save the generated code to `app/main_in_memory.py`.

In [2]:
# Prompt to generate a FastAPI app with in-memory storage that MUST match the provided SQL schema exactly.
in_memory_api_prompt = f"""
You are a senior Python developer.

Task: Generate a single-file FastAPI application (saved as app/main_in_memory.py) that uses simple in-memory
data structures (lists/dicts) for storage. This step validates the API surface and data shapes before wiring the real DB.

Strict rules (DO NOT BREAK):
1) DERIVE ALL FIELD NAMES FROM THE PROVIDED SQL SCHEMA: parse the provided `{sql_schema}` literally and use the
   exact column names in all Pydantic models, example payloads, and endpoint responses. If the schema shows
   `user_id`, `first_name`, `last_name`, use those names exactly (do NOT invent `id` or `full_name`).
2) TYPES MUST BE SQLITE-COMPATIBLE: map SQL types to Python types (INTEGER->int, VARCHAR/TEXT->str, DATE->date,
   TIMESTAMP/DATETIME->datetime). Provide appropriate example values in comments.
3) ENUMS & CHECKS: If the schema contains ENUM-like constraints or CHECK(...) clauses, represent them as simple
   string fields and include validation logic in Pydantic models (or Python Enums) that matches allowed values.
4) EXAMPLES: include at least one JSON example for each POST/PUT/PATCH in the code comments that exactly matches
   the schema's field names and types.
5) DO NOT produce database-migration code, dialect-specific types (Postgres ENUM), or change column names.

Output requirements:
- Single Python script content for `app/main_in_memory.py` as the final output of this prompt.
- Pydantic models' field names MUST match the SQL schema column names and types from `{sql_schema}`.
- CRUD endpoints for the 'users' table that use the exact field names, including example request bodies in comments.
- Include a small in-memory 'seed' list whose sample items respect the schema exactly.

SQL schema (context):\n```{sql_schema}```
"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    enhanced_in_memory_api_prompt = prompt_enhancer(in_memory_api_prompt)
    print("enhanced_in_memory_api_prompt:", enhanced_in_memory_api_prompt)
    generated_api_code = get_completion(enhanced_in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/main_in_memory.py", overwrite = True)
else:
    print("Skipping API generation because schema is missing.")

--- Generating FastAPI app with in-memory database ---


2025-10-30 11:39:36,215 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


enhanced_in_memory_api_prompt: <prompt>
  <persona>
    You are a senior Python backend engineer who specializes in rapid-prototyping REST APIs with FastAPI and Pydantic. You have deep knowledge of SQL→Python type-mapping, schema-driven model generation, and standards-compliant API design.
  </persona>

  <context>
    Goal: Produce a single-file FastAPI application that uses in-memory Python data structures (lists/dicts) instead of a database.  
    File path: app/main_in_memory.py

    Source of truth (SQL schema – DO NOT ALTER FIELD NAMES):
    ```
    CREATE TABLE users (
        user_id SERIAL PRIMARY KEY,
        first_name VARCHAR(50) NOT NULL,
        last_name VARCHAR(50) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE,
        role_id INT NOT NULL,
        start_date DATE NOT NULL,
        FOREIGN KEY (role_id) REFERENCES roles(role_id) ON DELETE CASCADE
    );

    CREATE TABLE roles (
        role_id SERIAL PRIMARY KEY,
        role_name VARCHAR(50) NOT NULL UNIQUE
   

### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Task:** Now, generate the specific SQLAlchemy code required to connect our application to the live `onboarding.db` SQLite database.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `sql_schema` as context again.
3.  Instruct the LLM to generate two separate pieces of code:
    * **SQLAlchemy Models:** Python classes that map to your database tables.
    * **Database Session Management:** The boilerplate code to create a database engine, session maker, and a dependency function (`get_db`) for use in FastAPI.
4.  The output should be two distinct, well-commented Python code blocks. We will integrate these manually in the next step.

In [3]:
# Prompt to generate SQLAlchemy ORM models and DB session code that exactly match the provided SQL schema.
db_code_prompt = f"""
You are a senior Python developer.

Task: Read the SQL schema below and produce TWO Python code blocks to be saved into `app/db_models.py`:\n
1) SQLAlchemy ORM model classes that use the exact column names and compatible types found in the schema. Do NOT
   rename columns (e.g., do not change `user_id` to `id`). If the schema uses separate `first_name` and `last_name`,
   the ORM model must reflect that. Map SERIAL -> Integer primary key with AUTOINCREMENT semantics where appropriate.
2) Database session/engine code: create_engine, SessionLocal, Base = declarative_base(), and a get_db dependency for FastAPI.

Important SQLite compatibility rules:\n
- Avoid dialect-specific types (Postgres ENUM/SERIAL). For enums use Python Enum stored as String or SQLAlchemy Enum
  with native_enum=False. For SERIAL, prefer Integer primary key and let SQLite autoincrement when inserting (or
  translate to INTEGER PRIMARY KEY AUTOINCREMENT in the example DDL).\n
- Use Date/DateTime mapping consistent with SQLite functions.\n

Output requirements:\n
- Return only raw Python source suitable for writing to `app/db_models.py`. The file must contain the ORM models (first),
  then the session/engine code.\n
- For each model, include a short comment showing the matching CREATE TABLE snippet from the given schema for clarity.\n
- Ensure column names and types match the provided `{sql_schema}` exactly so the ORM will not attempt to access columns that
  do not exist in the live DB.\n
\nSQL schema (context):\n```{sql_schema}```\n
"""

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    enhanced_db_code_prompt = prompt_enhancer(db_code_prompt)
    print("enhanced_db_code_prompt:", enhanced_db_code_prompt)
    generated_db_code = get_completion(enhanced_db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
    save_artifact(generated_db_code, "app/db_models.py", overwrite = True)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models and Session Code ---


2025-10-30 11:40:45,502 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


enhanced_db_code_prompt: <persona>
You are a Senior Python Backend Engineer with deep expertise in SQLAlchemy, FastAPI integration, and cross-database (especially SQLite) compatibility.
</persona>

<context>
You must translate the given SQL schema into SQLAlchemy ORM models and supporting database utilities.  
Key constraints & reminders:  
• Use the exact column names from the schema; never rename or merge columns.  
• Map SERIAL → Integer primary key with autoincrement semantics compatible with SQLite (i.e., Integer, primary_key=True, autoincrement=True).  
• For any enum/checked VARCHAR, model it as a Python Enum + SQLAlchemy Enum with native_enum=False so it also works on SQLite.  
• Avoid dialect-specific types; stick to SQLAlchemy core types (Integer, String(length), Text, Boolean, Date, DateTime).  
• Provide a brief “CREATE TABLE …” comment block directly above each corresponding model class for clarity.  
• After all models, create the database engine & session utilities:  
  

### Challenge 3 (Advanced): Integrating Live Database Logic

**Task:** This is the most critical engineering step of the lab. You will manually integrate the generated database code into the FastAPI application, replacing the in-memory logic with live database operations.

**Instructions:**
This task represents a significant jump in complexity. Follow these steps carefully in your IDE (like VS Code):

1.  Create a new, empty file named `app/main.py`.
2.  **First, copy the Pydantic models and the `app = FastAPI()` line** from your `app/main_in_memory.py` file and paste them into `app/main.py`.
3.  **Next, paste the SQLAlchemy model classes and the `get_db` dependency function** you generated in Challenge 2 into your new `app/main.py`.
4.  **Now, let's refactor the `POST /users/` endpoint.** Copy the endpoint function from the in-memory file, but replace the in-memory logic (e.g., `db.append()`) with the correct SQLAlchemy session calls: `db.add(db_user)`, `db.commit()`, and `db.refresh(db_user)`.
5.  Repeat this refactoring process for the other endpoints (GET, PUT, DELETE), replacing list manipulations with the appropriate SQLAlchemy `db.query()` methods.

This task requires you to act as the senior developer, stitching together the AI-generated components into a functional, cohesive whole. You may need to ask the LLM follow-up questions like, "How do I write a SQLAlchemy query to find a user by ID?"

In [4]:
# Integrate generated database code into a minimal live `app/main.py`
print("--- Integrating generated database code into final app/main.py ---")

# Load generated artifacts from the artifacts folder (app/main_in_memory.py and app/db_models.py)
in_memory_code = load_artifact("app/main_in_memory.py")
db_models_code = load_artifact("app/db_models.py")

integration_prompt = f"""
You are a senior developer. Based on the provided in-memory FastAPI
application and the SQLAlchemy models with database session code,
refactor the FastAPI application to use SQLAlchemy for database
interactions instead of in-memory data structures.

MANDATORY SAFETY RULES (must be followed exactly):
1) Field-name alignment: All Pydantic models, SQLAlchemy models, and
   endpoint code must use the exact column names from the provided
   `db_models_code` / `{sql_schema}`. Do NOT rename columns (e.g., do
   not change `user_id` to `id`).
2) Pydantic mapping: Set `Config.orm_mode = True` (or `Config.from_attributes = True` for
   Pydantic v2) in response models so ORM objects serialize correctly.
3) Query keys: When selecting by primary key, use the correct column
   name (e.g., `db.get(User, user_id)` or filter on `User.user_id`).
4) SQLite-safe types: Do not introduce dialect-specific types (Postgres ENUM/SERIAL).
5) Table creation: include `Base.metadata.create_all(engine)` only as a commented
   initialization option; do not automatically drop or alter existing tables.

TASK REQUIREMENTS:
- Produce a single complete Python source for `app/main.py` that:
  * Copies Pydantic models and the `app = FastAPI()` line from `app/main_in_memory.py`.
  * Inserts the SQLAlchemy model classes and `get_db` dependency from `app/db_models.py`.
  * Refactors POST/GET/PUT/PATCH/DELETE endpoints to use SQLAlchemy session calls
    (db.add, db.commit, db.refresh, db.query, db.get) while using the exact column names.
- Return only the raw Python source for `app/main.py` (no extra commentary outside the code).

Context blocks for reference:\n```python\n{in_memory_code}\n```\n```python\n{db_models_code}\n```
"""

enhanced_integration_prompt = prompt_enhancer(integration_prompt)
print("enhanced_integration_prompt:", enhanced_integration_prompt)
generated_integration_code = get_completion(enhanced_integration_prompt, client, model_name, api_provider)
cleaned_integration_code = clean_llm_output(generated_integration_code, language='python')

print("--- Final integrated app (preview) ---")
print(cleaned_integration_code)
save_artifact(cleaned_integration_code, "app/main.py", overwrite = True)
print("--- Saved integrated app to app/main.py ---")

# Expose final_api_code for downstream cells that may reference it
code_final_api = cleaned_integration_code
print("--- Final Integrated API code for app/main.py ---")
print(code_final_api)
save_artifact(code_final_api, "app/main.py", overwrite = True)

--- Integrating generated database code into final app/main.py ---


2025-10-30 11:41:47,042 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


enhanced_integration_prompt: <persona>
You are an elite Senior Python Backend Engineer with deep expertise in FastAPI, SQLAlchemy, and Pydantic.  
Think step-by-step internally to ensure perfect alignment with the database schema, safety rules, and FastAPI best practices, but do NOT expose your reasoning—output only the final code file.
</persona>

<context>
1. You are given:  
   • The full in-memory FastAPI application (`app/main_in_memory.py`).  
   • Complete SQLAlchemy ORM definitions and `get_db` dependency (`app/db_models.py`).  

2. Mandatory Safety Rules (must be followed exactly):  
   a. Field-name alignment: All Pydantic models, SQLAlchemy models, and endpoint code must use the exact column names from the provided DDL (e.g., `user_id`, `role_id`).  
   b. Set `Config.orm_mode = True` (or `from_attributes = True` in Pydantic v2) for all response models.  
   c. When querying by primary key, use the correct column name (e.g., `db.get(User, user_id)` or `.filter(User.user_id =

WindowsPath('C:/Users/labadmin/Documents/AG-AISOFTDEV/artifacts/app/main.py')

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.